## Have a Comments - Consolida Lista de Documentos:

### Importa Bibliotecas

In [1]:
import pandas as pd
import re
import numpy as np
from pathlib import Path
import glob
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")

### Carrega Dados Básicos

In [2]:
# Set the correct path to the Excel file
excel_filename = 'C:/Users/ELXY/Documents/Codigos/Python/P84_85/DadosBasicos/dados_basicos.xlsx'

# Read the Excel file
df_db = pd.read_excel(excel_filename, sheet_name=0)  # sheet_name=0 reads the first sheet
df_db = df_db.fillna('')
# display(df_db)

### Renomeia Colunas Duplicadas

In [3]:
def renomear_colunas_revisao(df):
    """
    Renomeia colunas duplicadas em um DataFrame usando prefixos de revisão (rev0_, revA_, revB_, etc.)
    
    Parâmetros:
        df (pandas.DataFrame): DataFrame com possíveis nomes de colunas duplicados
        
    Retorna:
        pandas.DataFrame: DataFrame com nomes de colunas únicos
    """
    # Cria uma cópia do DataFrame para não modificar o original
    df_renomeado = df.copy()
    
    # Identificando as colunas duplicadas e suas posições
    posicoes = {}
    for i, col in enumerate(df.columns):
        if col not in posicoes:
            posicoes[col] = []
        posicoes[col].append(i)
    
    # Lista de prefixos de revisão
    prefixos_revisao = ['rev0_', 'revA_', 'revB_', 'revC_', 'revD_', 'revE_', 
                         'revF_', 'revG_', 'revH_', 'revI_', 'revJ_']
    
    # Criando novos nomes com os prefixos de revisão
    novos_nomes = []
    for i, col in enumerate(df.columns):
        if len(posicoes[col]) > 1:
            # Encontra o índice desta ocorrência na lista de posições
            idx = posicoes[col].index(i)
            if idx < len(prefixos_revisao):
                novos_nomes.append(f'{prefixos_revisao[idx]}{col}')
            else:
                # Caso tenha mais revisões que prefixos disponíveis
                novos_nomes.append(f'rev{idx}_{col}')
        else:
            # Colunas sem duplicatas mantêm o nome original
            novos_nomes.append(col)
    
    # Atribui os novos nomes às colunas
    df_renomeado.columns = novos_nomes
    
    return df_renomeado

In [4]:
# Caminho base para os arquivos
base_path = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\files\\'
# Arquivo do relatório Integra
integra_report = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\RE-General Query - Technical Engineering Documents.xlsx'

# Dataframe para armazenar todos os resultados
df_LD_final = pd.DataFrame()

df_integra = pd.read_excel(integra_report, sheet_name="DADOS")

# Iterar sobre cada linha do dataframe
for _, row in df_db.iterrows():
    ld_number = row['LD_Number']
    package = row['PKG_DESCRIPTION']

    if ld_number:
        print(f'LD Number: {ld_number}, processando arquivos do pacote: {package}...')
        ld_files = glob.glob(f'{base_path}{ld_number}*.xlsx')
        
        if len(ld_files) < 2:
            print(f"Não foram encontrados pelo menos 2 arquivos para o pacote {package}")
            continue

        ld_file_comments = [f for f in ld_files if '_comments' in f][0]

        # Carregar os dataframes 
        df_LD_comments = pd.read_excel(ld_file_comments, sheet_name="VDRL")
    
        df_LD_comments.columns = df_LD_comments.iloc[6]
        df_LD_comments = df_LD_comments.iloc[7:]
    
        df_LD_comments = renomear_colunas_revisao(df_LD_comments)

        df_LD_comments.rename(columns = {"Have comment?":"Comments", "CLIENT DOCUMENT NUMBER":"CLIENT_DOCUMENT",
                                "VENDOR DOCUMENT NUMBER":"VENDOR_DOCUMENT",
                                "DOCUMENT TITLE": "DOCUMENT_TITLE"}, inplace = True)
        
        df_LD_comments = df_LD_comments.dropna(subset=['DOCUMENT_TITLE'])
        # Remover espaços em branco e converter para maiúsculas
        df_LD_comments['DOCUMENT_TITLE'] = df_LD_comments['DOCUMENT_TITLE'].apply(lambda x: x.strip().upper())
    
        df_integra.rename(columns = {"CREATION DATE":"CREATION_DATE"}, inplace = True)

        colunas_para_manter = ['Comments', 'Discipline', 'CLIENT_DOCUMENT', 'VENDOR_DOCUMENT', 'DOCUMENT_TITLE', 'ORIGINATOR ', 'rev0_PLANNED DATE','rev0_ACTUAL DATE',
                                'rev0_RETURNED DATE','rev0_RETURNED CODE','revA_PLANNED DATE','revA_ACTUAL DATE','revA_RETURNED DATE','revA_RETURNED CODE', 
                                'revB_PLANNED DATE','revB_ACTUAL DATE','revB_RETURNED DATE','revB_RETURNED CODE']
        df_LD_comments.drop(columns=[col for col in df_LD_comments.columns if col not in colunas_para_manter], inplace = True)
        df_LD_comments.reset_index(drop=True, inplace = True)
    
        df_LD_comments = df_LD_comments.iloc[:, :-2] 

        # Get only the needed columns from df_integra
        code_mapping = df_integra[['CODE', 'TITLE', 'CREATION_DATE']].copy()
        # Create a dictionary for faster lookups
        code_dict = dict(zip(code_mapping['CODE'], zip(code_mapping['TITLE'], code_mapping['CREATION_DATE'])))
        # Create new columns all at once using map
        df_LD_comments['CODE'] = df_LD_comments['CLIENT_DOCUMENT'].map(lambda x: x if x in code_dict else None)
        df_LD_comments['TITLE'] = df_LD_comments['CLIENT_DOCUMENT'].map(lambda x: code_dict.get(x, (None,None))[0])
        df_LD_comments['CREATION_DATE'] = df_LD_comments['CLIENT_DOCUMENT'].map(lambda x: code_dict.get(x, (None,None))[1])
        df_LD_comments['PKG_DESCRIPTION'] = package

        # Adicionar ao dataframe final
        df_LD_final = pd.concat([df_LD_final, df_LD_comments], ignore_index=True)
    else:
        print(f"LD Number vazia. Pulando...")
        continue

LD Number: I-LD-3010.2S-1200-940-KD9-001_0, processando arquivos do pacote: API 610 CENTRIFUGAL PUMPS...
LD Number vazia. Pulando...
LD Number vazia. Pulando...
LD Number: I-LD-3010.2S-5122-940-D5A-001_B, processando arquivos do pacote: FRESH WATER CHLORINATION UNIT...
LD Number: I-LD-3010.2S-5241-940-GK1-001_A, processando arquivos do pacote: NITROGEN GENERATOR UNITS...
LD Number: I-LD-3010.2S-120N-940-KD9-001_0, processando arquivos do pacote: NON-API 610 CENTRIFUGAL PUMPS...
LD Number: I-LD-3010.2S-1210-911-AP5-001_A, processando arquivos do pacote: PIG LAUNCHERS AND RECEIVERS...
LD Number: I-LD-3010.2S-5412-911-NZA-001_A, processando arquivos do pacote: PROGRESSIVE CAVITY PUMPS...
LD Number: I-LD-3010.2S-5000-940-KFQ-001_0, processando arquivos do pacote: RECIPROCATING PUMPS...
LD Number: I-LD-3010.2S-5121-940-D5A-001_A, processando arquivos do pacote: SEA WATER ELECTROCHLORINATION UNIT...
LD Number: I-LD-3010.2S-5111-940-FM2-001_B, processando arquivos do pacote: SEA WATER LIFT PU

### Cria uma coluna 'PLANNED DATE' com os valores mais atualizados da LD e Reordena a LD_final

In [5]:
def atualizar_planned_date_coalesce(df_LD):
    """
    Versão usando coalesce para criar uma coluna 'PLANNED DATE' com os valores mais atualizados,
    priorizando a revisão mais recente com valor válido.
    """
    # Cria uma cópia do DataFrame
    df = df_LD.copy()
    
    # Identifica as colunas de PLANNED DATE em ordem de prioridade
    colunas_planned_date = [col for col in df.columns if 'PLANNED DATE' in col]
    colunas_planned_date.sort(reverse=True)  # Coloca em ordem: revB, revA, rev0
    
    # Usa o método coalesce do pandas para pegar o primeiro valor não-NaN
    df['PLANNED_DATE'] = df[colunas_planned_date].bfill(axis=1).iloc[:, 0]

    # Definimos a nova ordem das colunas em uma lista
    new_order = ['PKG_DESCRIPTION','CLIENT_DOCUMENT','VENDOR_DOCUMENT','DOCUMENT_TITLE','rev0_PLANNED DATE','rev0_ACTUAL DATE','rev0_RETURNED DATE','rev0_RETURNED CODE','revA_PLANNED DATE','revA_ACTUAL DATE','revA_RETURNED DATE','revA_RETURNED CODE','revB_PLANNED DATE','revB_ACTUAL DATE','CODE','TITLE','CREATION_DATE','PLANNED_DATE','Comments','Discipline']

    return df[new_order]

df_LD_final = atualizar_planned_date_coalesce(df_LD_final)


#### Lista os documentos que não batem a descrição e grava no arquivo "have_a_comment_mismatches.xlsx".

In [6]:
# Create mask for non-matching titles
mask = (df_LD_final['DOCUMENT_TITLE'] != df_LD_final['TITLE']) & (~df_LD_final['TITLE'].isna())

# Get mismatched documents
mismatches = df_LD_final[mask][['CODE', 'CLIENT_DOCUMENT', 'DOCUMENT_TITLE', 'TITLE']]

print("Documents with mismatching titles:")
display(mismatches)

# Salvar o DataFrame final
output_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\have_a_comment_mismatches.xlsx'

with pd.ExcelWriter(output_file) as writer:
    mismatches.to_excel(writer, sheet_name='mismatches', index=False)

print(f'Arquivo {output_file} salvo em: ', Path.cwd())

Documents with mismatching titles:


,CODE,CLIENT_DOCUMENT,DOCUMENT_TITLE,TITLE
1,I-PR-3010.2S-1200-974-KD9-001,I-PR-3010.2S-1200-974-KD9-001,PRESSURE TEST PROCEDURES - API 610 & NON-API ...,PRESSURE TEST PROCEDURES - API 610 AND NON-API...
3,I-PR-3010.2S-1200-956-KD9-001,I-PR-3010.2S-1200-956-KD9-001,SURFACE PREPARATION AND PAINTING PROCEDURE - ...,SURFACE PREPARATION AND PAINTING PROCEDURE - A...
4,I-PR-3010.2S-1200-972-KD9-001,I-PR-3010.2S-1200-972-KD9-001,PMI PROCEDURE - API 610 & NON-API CENTRIFUGAL ...,PMI PROCEDURE - API 610 AND NON-API CENTRIFUGA...
5,I-PR-3010.2S-1200-972-KD9-002,I-PR-3010.2S-1200-972-KD9-002,NON-DESTRUCTIVE EXAMINATION PROCEDURE - API 61...,NON-DESTRUCTIVE EXAMINATION PROCEDURE - API 61...
14,I-PR-3010.2S-1200-972-KD9-003,I-PR-3010.2S-1200-972-KD9-003,HARDNESS PROCEDURE - API 610& NON-API CENTRIF...,HARDNESS PROCEDURE - API 610AND NON-API CENTRI...
...,...,...,...,...
3162,I-DE-3010.2S-5266-946-ND1-001,I-DE-3010.2S-5266-946-ND1-001,INTERCONNECT DIAGRAM EXTERNAL CONNECTIONS FOR ...,INTERCONNECT DIAGRAM EXTERNAL CONNECTIONS FOR ...
3163,I-DE-3010.2S-5266-946-ND1-002,I-DE-3010.2S-5266-946-ND1-002,INTERCONNECT DIAGRAM EXTERNAL CONNECTIONS FOR ...,INTERCONNECT DIAGRAM EXTERNAL CONNECTIONS FOR ...
3165,I-DE-3010.2S-5266-946-ND1-004,I-DE-3010.2S-5266-946-ND1-004,SINGLE LINE DIAGRAM ELECTRICAL SYSTEMS_AFT PED...,SINGLE LINE DIAGRAM ELECTRICAL SYSTEMS-AFT PED...
3166,I-DE-3010.2S-5266-946-ND1-005,I-DE-3010.2S-5266-946-ND1-005,SINGLE LINE DIAGRAM ELECTRICAL SYSTEMS_FWD PED...,SINGLE LINE DIAGRAM ELECTRICAL SYSTEMS-FWD PED...


Arquivo C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\have_a_comment_mismatches.xlsx salvo em:  c:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs


### Salvar o DataFrame final

In [7]:
# Salvar o DataFrame final
output_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_consolidada_have_a_comment.xlsx'

with pd.ExcelWriter(output_file) as writer:
    df_LD_final.to_excel(writer, sheet_name='VDRL', index=False)

print(f'Arquivo {output_file} salvo em: ', Path.cwd())

Arquivo C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\LD_consolidada_have_a_comment.xlsx salvo em:  c:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs


### Carrega Have a Comments

In [8]:
import pandas as pd
from pathlib import Path
import warnings
from datetime import datetime

warnings.filterwarnings("ignore", category=FutureWarning)
intput_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_consolidada_have_a_comment.xlsx'

df_LD_final = pd.read_excel(intput_file, sheet_name="VDRL")

### Consolida todas as VDAs

In [9]:
import os
import pandas as pd
from pathlib import Path

# Caminho da pasta contendo os arquivos VDA 
vda_path = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\VDA\\'

# Caminho do arquivo consolidado
output_path = os.path.join(vda_path, 'vda.xlsx')

# Verificar se o arquivo consolidado já existe e excluí-lo
if os.path.exists(output_path):
    try:
        os.remove(output_path)
        print(f"Arquivo existente {output_path} foi excluído com sucesso.")
    except Exception as e:
        print(f"Erro ao excluir o arquivo {output_path}: {e}")
        print("O arquivo pode estar aberto em outro programa. Feche-o e tente novamente.")
        exit()  # Encerrar o script se não conseguir excluir o arquivo

# Lista para armazenar todos os dataframes 
all_dfs = [] 

# Iterar sobre todos os arquivos Excel na pasta 
for filename in os.listdir(vda_path): 
    # Ignorar arquivos temporários/ocultos que começam com ~$ ou .
    if filename.startswith('~$') or filename.startswith('.'):
        print(f"Ignorando arquivo temporário: {filename}")
        continue
        
    if filename.endswith('.xlsx'): 
        file_path = os.path.join(vda_path, filename) 
        print(f"Lendo arquivo: {file_path}")
        try:
            # Especificar o engine como 'openpyxl' para arquivos .xlsx
            df = pd.read_excel(file_path, engine='openpyxl') 
            all_dfs.append(df)
        except Exception as e:
            print(f"Erro ao ler {file_path}: {e}")
            
    elif filename.endswith('.xls'):
        file_path = os.path.join(vda_path, filename) 
        print(f"Lendo arquivo: {file_path}")
        try:
            # Especificar o engine como 'xlrd' para arquivos .xls
            df = pd.read_excel(file_path, engine='xlrd') 
            all_dfs.append(df)
        except Exception as e:
            print(f"Erro ao ler {file_path}: {e}")

# Verificar se encontramos algum arquivo
if all_dfs:
    # Concatenar todos os dataframes 
    consolidated_df = pd.concat(all_dfs, ignore_index=True) 
    # Filter the dataframe to keep only system 1223 for HM9 trigram, and keep all data for other trigrams
    consolidated_df = consolidated_df[
        ((consolidated_df['Reference Document'].str.contains('-HM9-', na=False)) & 
        (consolidated_df['Reference Document'].str.contains('-1223-', na=False))) |
        (~consolidated_df['Reference Document'].str.contains('-HM9-', na=False))
]
    # Salvar o dataframe consolidado e filtrado como um novo arquivo Excel 
    consolidated_df.to_excel(output_path, index=False) 
    print(f'Arquivo consolidado salvo em: {output_path}')
else:
    print("Nenhum arquivo Excel válido encontrado no diretório.")


Arquivo existente C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\vda.xlsx foi excluído com sucesso.
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\All_VDA.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA AP5 14.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA ATI 19.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA D5A 65.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA D5A 66.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA FM2 36.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA GK1 41.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA HE1 21.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA HLT 06.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\VDA\VDA HM9 26.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos

### Relaciona o documento com a to-do-list

In [10]:
import pandas as pd
import os
from openpyxl import load_workbook
from datetime import datetime

# Caminhos dos arquivos 
todo_list_path = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\To Do List.xlsx'
vda_path = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\VDA\\vda.xlsx'

# Leitura dos arquivos 
vda_df = pd.read_excel(vda_path)

# Tentar ler o arquivo To Do List com diferentes métodos 
try:
    # Primeiro, tente ler normalmente 
    todo_df = pd.read_excel(todo_list_path, engine='openpyxl')
except Exception as e:
    print(f"Erro ao ler o arquivo normalmente: {e}")
    try:
        # Se falhar, tente ler apenas os dados 
        wb = load_workbook(filename=todo_list_path, read_only=True, data_only=True)
        ws = wb.active
        data = ws.values
        cols = next(data)[0:]
        todo_df = pd.DataFrame(data, columns=cols)
    except Exception as e:
        print(f"Erro ao ler o arquivo com openpyxl: {e}")
        # Se ambos falharem, crie um DataFrame vazio
        todo_df = pd.DataFrame(columns=['Related Object'])

# Verificar se a coluna 'Related Object' existe
if 'Related Object' not in todo_df.columns:
    print("A coluna 'Related Object' não foi encontrada. Adicionando uma coluna vazia.")
    todo_df['Related Object'] = ''

# Resto do código permanece o mesmo 
# Merge do vda_df com df_LD_final 
merged_df = pd.merge(vda_df, df_LD_final, left_on='Reference Document', right_on='CLIENT_DOCUMENT', how='left')

# Função para criar a string de informações 
def create_info_string(row):
    if pd.notna(row['CLIENT_DOCUMENT']):
        # Obter a data atual no formato desejado
        current_date = datetime.now().strftime("%d-%B-%Y")
        # Limpar e padronizar o valor de Comments
        have_comment = str(row['Comments']).strip().upper()

        if have_comment == 'YES':
            return f"DOCUMENT: {row['CLIENT_DOCUMENT']} {row['Title']}, Comment: {row['Comments']}, Discipline: {row['Discipline']}, Date: {current_date}"
        else:
            return f"DOCUMENT: {row['CLIENT_DOCUMENT']} {row['Title']}, Comment: NO, Date: {current_date}"
    else:
        return ""

# Aplicar a função para criar a coluna de informações 
merged_df['Info'] = merged_df.apply(create_info_string, axis=1)

# Criar um dicionário de Code para Info
code_to_info = dict(zip(merged_df['Code'], merged_df['Info']))

# Função para preencher a coluna Additional Info
def fill_additional_info(code):
    return code_to_info.get(code, '')

# Adicionar a nova coluna Additional Info no todo_df 
todo_df['Additional Info'] = todo_df['Related Object'].apply(fill_additional_info)

# Update Additional Info based on Task Name condition
todo_df.loc[todo_df['Task Name'] != 'VDA have a Comments', 'Additional Info'] = 'NÃO É HAVE A COMMENT'

# Salvar o arquivo atualizado
todo_df.to_excel(todo_list_path, index=False, engine='openpyxl')

print(f'Arquivo To Do List atualizado e salvo em: {todo_list_path}')

Arquivo To Do List atualizado e salvo em: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\To Do List.xlsx
